In [3]:
import tensorflow as tf
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


using the famous FER-2013 as dataset to train and test model

In [4]:

Datadirectory='/content/drive/MyDrive/ML_Project/train/'
# path=os.path.join(Datadirectory,"")
# print(path)
Classes=['happy','neutral','sad']


Changing the size of image from 48X48 to 224X224 to easily put it in the model(now doing it only for whole dataset)

In [5]:
training_data=[]
def create_training_data():
  for Category in Classes:
    path=os.path.join(Datadirectory,Category)
    class_num=Classes.index(Category)
    for img in os.listdir(path):
      img_arr=cv2.imread(os.path.join(path,img))
      new_array=cv2.resize(img_arr,(224,224))
      training_data.append([new_array,class_num])

create_training_data()


In [6]:
import random
random.shuffle(training_data)
X=[]
Y=[]
for features,label in training_data:
  X.append(features)
  Y.append(label)
X=np.array(X).reshape(-1,224,224,3) #changing dimensions to properly allow model to work



In [7]:
X=X/255 #normalizing data

In [8]:
print(len(X),len(X[0]),len(X[0][0]),(X[0][0][0]))

2400 224 224 [0.88235294 0.88235294 0.88235294]


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
model=tf.keras.applications.MobileNetV2() #use MobileNetV2 archutecture pre-training model, as it being light weight we can use it for live demo



14536120/14536120 [==============================] - 0s 0us/step


Doing transfer learning to make the model learn classes from last step, and hence we won't have to train the whole model from start, and we can use the already existing weights and biases

In [10]:
base_input=model.layers[0].input
base_output=model.layers[-2].output #removes the last layer

#  Adding more layers at end of model to perform classification of classes
final_output=layers.Dense(128)(base_output)
final_output=layers.Activation('relu')(final_output)
final_output=layers.Dense(64)(final_output)
final_output=layers.Activation('relu')(final_output)
final_output=layers.Dense(3,activation='softmax')(final_output)

In [11]:
X=np.array(X)
Y=np.array(Y)

In [ ]:
new_model=keras.Model(inputs=base_input,outputs=final_output)
for layer in new_model.layers[:-4]:
    layer.trainable = False
new_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
new_model.fit(X,Y,epochs=100) 


In [ ]:
new_model.save('model.h5')

In [ ]:
model=tf.keras.models.load_model('/content/model.h5')

In [ ]:
img_1=cv2.imread("/content/drive/MyDrive/ML_Project/train/angry/Training_20748658.jpg")
img_1=cv2.resize(img_1,(224,224))
img_1=np.array(img_1).reshape(-1,224,224,3)
print(img_1.shape)


(1, 224, 224, 3)


In [ ]:
import pandas as pd
predicted = pd.DataFrame()
predicted_1=model.predict(img_1)
predicted["happy"]=predicted_1[:,0]
predicted["neutral"]=predicted_1[:,1]
predicted["sad"]=predicted_1[:,2]
predicted.head()